# Рад са моделима из Meta породице

## Увод

Ова лекција обухвата:

- Истраживање два главна модела из Meta породице - Llama 3.1 и Llama 3.2
- Разумевање намена и сценарија за сваки модел
- Пример кода који показује јединствене карактеристике сваког модела

## Meta породица модела

У овој лекцији ћемо истражити 2 модела из Meta породице или "Llama Herd" - Llama 3.1 и Llama 3.2

Ови модели постоје у различитим варијантама и доступни су на Github Model marketplace-у. Више детаља о коришћењу Github Models за [прототипирање са AI моделима](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Варијанте модела:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Напомена: Llama 3 је такође доступан на Github Models, али неће бити обрађен у овој лекцији*


## Ллама 3.1

Са 405 милијарди параметара, Ллама 3.1 спада у категорију отвореног кода LLM модела.

Ова верзија је унапређена у односу на претходну Ллама 3 и доноси:

- Већи контекстуални прозор – 128k токена уместо 8k токена
- Већи максимални број излазних токена – 4096 уместо 2048
- Бољу подршку за више језика – захваљујући већем броју токена у обуци

Ове могућности омогућавају Ллама 3.1 да обрађује сложеније случајеве употребе при развоју GenAI апликација, укључујући:
- Нативно позивање функција – могућност позивања спољних алата и функција ван LLM радног тока
- Боље RAG перформансе – захваљујући већем контекстуалном прозору
- Генерисање синтетичких података – могућност креирања ефикасних података за задатке као што је фино подешавање


### Позивање нативних функција

Llama 3.1 је додатно прилагођена да буде ефикаснија у позивању функција или алата. Такође има два уграђена алата које модел може да препозна као потребне за коришћење на основу упита корисника. Ови алати су:

- **Brave Search** – Може се користити за добијање најновијих информација као што је временска прогноза путем претраге на интернету
- **Wolfram Alpha** – Може се користити за сложеније математичке прорачуне, тако да није потребно да сами пишете функције.

Можете такође направити сопствене прилагођене алате које LLM може да позове.

У примеру кода испод:

- Дефинишемо доступне алате (brave_search, wolfram_alpha) у системском упиту.
- Пошаљемо кориснички упит који пита за временску прогнозу у одређеном граду.
- LLM ће одговорити позивом алата Brave Search који ће изгледати овако `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Напомена: Овај пример само позива алат, ако желите да добијете резултате, потребно је да направите бесплатан налог на Brave API страници и дефинишете саму функцију*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Ллама 3.2

Иако је Ллама 3.1 велики језички модел, једно од њених ограничења је мултимодалност. То значи да не може да користи различите типове улаза као што су слике као упите и да на њих одговара. Ова способност је једна од главних карактеристика Ллама 3.2. Ове могућности такође укључују:

- Мултимодалност – има могућност да обрађује и текстуалне и сликовне упите
- Варијанте мале и средње величине (11B и 90B) – ово омогућава флексибилне опције за имплементацију,
- Варијанте само за текст (1B и 3B) – ово омогућава моделу да се користи на edge / мобилним уређајима и обезбеђује ниску латенцију

Подршка за мултимодалност представља велики корак у свету open source модела. Пример кода испод прихвата и слику и текстуални упит како би добио анализу слике од Ллама 3.2 90B.

### Мултимодална подршка са Ллама 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Учење се не завршава овде, наставите своје путовање

Након што завршите ову лекцију, погледајте нашу [Збирку за учење о генеративној вештачкој интелигенцији](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) како бисте наставили да унапређујете своје знање о генеративној вештачкој интелигенцији!



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем AI услуге за превођење [Co-op Translator](https://github.com/Azure/co-op-translator). Иако настојимо да обезбедимо тачност, имајте у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на изворном језику треба сматрати меродавним извором. За критичне информације препоручује се професионални људски превод. Не сносимо одговорност за било каква неспоразума или погрешна тумачења настала коришћењем овог превода.
